#**FDP-HITK - April 23, 2024. Hands-on Session**

# **Part:1** - Use a hosted LLM - Anthropic Claude-3

# Anthropic

Anthropic has recently released its latest models: `Claude 3 Opus`, `Claude 3 Sonnet`, and `Claude 3 Haiku` (which will be available soon).
By default, the `claude-2.1 model` is used.
In this notebook we will utilize **Claude 3 Opus** model [claude-3-opus-20240229]


# Installation

In [28]:
# Installs successfully....

!pip install llama-index-llms-anthropic

!pip install llama-index

## Import Libraries

In [1]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

## Set Tokenizer

First we want to set the tokenizer, which is slightly different than TikToken.

**NOTE**: The Claude 3 tokenizer has not been updated yet; using the existing Anthropic tokenizer leads to context overflow errors for 200k tokens. We've temporarily set the max tokens for Claude 3 to 180k.

In [2]:
tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

## Set the Anthropic API Key

In [7]:
import os
os.environ["ANTHROPIC_API_KEY"] = "<your Anthropic API Key goes here>"


## Initialize the LLM

In [8]:
from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model="claude-3-opus-20240229")


In [5]:
print(type(llm))

<class 'llama_index.llms.anthropic.base.Anthropic'>


In [36]:
# llm.dict()

## Put a Query, Claude-3 answers from its Knowledgebase

#### Call `complete` with a prompt

In [9]:
resp = llm.complete("Rihard Cleyderman is ")
print(resp)

Richard Clayderman is a French pianist known for his instrumental renditions of popular music and film soundtracks. Some key facts about him:

1. Born Philippe Pagès on December 28, 1953, in Paris, France.

2. He adopted the stage name "Richard Clayderman" at the beginning of his career.

3. Clayderman has released numerous albums and has sold over 150 million records worldwide, making him one of the best-selling French musicians of all time.

4. His debut album, "Ballade pour Adeline," released in 1977, was a major success and included his signature piece, a instrumental version of the song "Ballade pour Adeline."

5. He is known for his romantic, easy-listening style and has covered a wide range of music, from classical to pop, film soundtracks, and folk music.

6. Clayderman has performed in numerous concerts around the world and has collaborated with various orchestras and musicians throughout his career.

7. In addition to his music career, he is also involved in various charitabl

In [ ]:
resp = llm.complete("Who is Sachin Tendulkar?  What is his notable achievements.")
resp.text

#### Call `chat` with a list of messages

In [ ]:
from llama_index.core.llms import ChatMessage

messages = [
    ChatMessage(
        role="system",
        content="You are a helpful personal assistant. Answer the questions of the users beirfly. If you do not know the answer of a question, reply saying that you do not know the answer."
    ),
    ChatMessage(role="user",
                content="Tell me about the battle of Panipat."
                ),
]


In [ ]:
resp = llm.chat(messages)

print(resp)

# print(resp.message)

In [ ]:
resp = llm.complete("What is self-attention defined in the paper Attention is all you need?")
resp.text

In [ ]:
resp = llm.complete("What are the major technical ideas described in the first Yolo paper?")
resp.text

In [ ]:
resp = llm.complete("What are the major technical ideas described in the YoloXNet paper?")
resp.text

In [ ]:
resp = llm.complete("What does the term elderbo-defacto mean?")
resp.text

# **++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++**

# **Build a RAG Application**

## Installations

In [ ]:
!pip install transformers==4.30
!pip install llama_index

!pip install pypdf  # Reading PDF files

In [27]:
## Embedding
!pip install langchain
!pip install sentence_transformers
!pip install llama-index-embeddings-langchain

## Necessary Import

In [22]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, ServiceContext
from llama_index.core.prompts.prompts import SimpleInputPrompt

## Load Documents from the Data Folder

In [23]:
documents = SimpleDirectoryReader("/content/data").load_data()

print(type(documents))
print(len(documents)) # 25 pages total (attention.pdf has 15 pages, yolo.pdf has 10 pages)

<class 'list'>
4


### Browse the Documents

In [ ]:
# print(documents[0])

print(documents[0].id_)
print(documents[0].text)

In [ ]:
print(documents[1].id_)
print(documents[1].text)

In [ ]:
print(documents[2].id_)
print(documents[2].text)

In [ ]:
print(documents[3].id_)
print(documents[3].text)

## Login to Huggingface with CLI

In [24]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
Your token has been saved to /root/.cache/huggingface/token
Login successful


## Import necessary Libraries for Embeddings

We are using **all-MiniLM-L6-v2** model from Huggingface to generate vector embeddings of dimension 384.


In [29]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index.embeddings.langchain import LangchainEmbedding

embed_model = LangchainEmbedding (
    HuggingFaceEmbeddings (
        model_name = "sentence-transformers/all-MiniLM-L6-v2"  # generates 384 dimensional vectors
        )
    )

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Test Vector Embeddings

In [ ]:
embed1 = embed_model.get_query_embedding("What a beautiful day.")
print(len(embed1))
print(embed1[:10])

# embed2 = embed_model.get_query_embedding("What a wonderful day.")
embed2 = embed_model.get_query_embedding("What a bad day.")

print(len(embed2))
print(embed2[:10])

import numpy as np
from scipy.spatial import distance

print(distance.cosine(embed1, embed2))
print(distance.euclidean(embed1, embed2))
print(np.dot(embed1, embed2))


# print(distance.cosine(embed1, embed1))
# print(distance.euclidean(embed1, embed1))
# np.dot(embed1, embed1)


## Set the System Prompt

In [30]:
system_prompt="""
You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided.
"""
## Default format supportable by LLama2 (Let's try the same for Anthropic LLM)
query_wrapper_prompt = SimpleInputPrompt("<|USER|>{query_str}<|ASSISTANT|>")

## Create a Service Context

In [31]:
from llama_index.core import ServiceContext

service_context = ServiceContext.from_defaults (
    chunk_size = 1024,
    llm = llm,
    embed_model = embed_model,
    system_prompt = system_prompt,
    query_wrapper_prompt = query_wrapper_prompt
)

service_context.to_dict()


<ipython-input-31-af9494adeacd>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  service_context = ServiceContext.from_defaults (


{'llm': {'system_prompt': '\nYou are a Q&A assistant. Your goal is to answer questions as\naccurately as possible based on the instructions and context provided.\n',
  'pydantic_program_mode': <PydanticProgramMode.DEFAULT: 'default'>,
  'model': 'claude-3-opus-20240229',
  'temperature': 0.1,
  'max_tokens': 512,
  'base_url': None,
  'timeout': None,
  'max_retries': 10,
  'additional_kwargs': {},
  'class_name': 'Anthropic_LLM'},
 'llm_predictor': {'system_prompt': '\nYou are a Q&A assistant. Your goal is to answer questions as\naccurately as possible based on the instructions and context provided.\n',
  'query_wrapper_prompt': {'metadata': {'prompt_type': <PromptType.CUSTOM: 'custom'>},
   'template_vars': ['query_str'],
   'kwargs': {},
   'output_parser': None,
   'template_var_mappings': None,
   'function_mappings': None,
   'template': '<|USER|>{query_str}<|ASSISTANT|>'},
  'pydantic_program_mode': <PydanticProgramMode.DEFAULT: 'default'>,
  'class_name': 'LLMPredictor',
  'llm

## Create the Embeddings from the Doduments and Store in the VectorStore (also perform indexing)

In [32]:
index = VectorStoreIndex.from_documents(documents,
                                        service_context = service_context)
index

## Define Query Engine and Chat Engine

In [33]:
query_engine = index.as_query_engine()

In [36]:
chat_engine = index.as_chat_engine()

## Pose Queries

In [34]:
response = query_engine.query("what is attention is all you need?")
print(response.response)

I apologize, but the given context does not contain any information about the paper "Attention is All You Need". The context discusses a model called YoloXNet which is a hybrid CNN-LSTM architecture for image captioning. It does not mention anything related to the "Attention is All You Need" paper or the Transformer architecture it introduced.


In [38]:
response = chat_engine.chat("what is attention is all you need?")
print(response.response)

I'm sorry, but I don't have enough information to provide a detailed answer about the "Attention is All You Need" paper. The knowledge base I have access to appears to be focused on computer vision models like YoloXNet and does not contain the relevant details needed to explain what the "Attention is All You Need" paper is about or the Transformer architecture it introduced for natural language processing tasks. I would need additional resources with more information on this influential NLP paper and model to be able to give you a satisfactory answer to your question.


----------------------------------------------------------------------

In [ ]:
response = query_engine.query("Tell me about the YOLO model in deep learning?")

print(response.response)

In [ ]:
response = chat_engine.chat("Tell me about the YOLO model in deep learning?")
print(response.response)

----------------------------------------------------------------------

In [ ]:
response = query_engine.query("what is YoloXNet?")
print(response.response)

In [ ]:
response = chat_engine.chat("what is YoloXNet?")
print(response.response)

# +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++

# **PART-3** : Connecting with a Hosted VectorStore - Pinecone

# Try Pinecone as the VectorStore

In [2]:
# Installs successfulle....
!pip install pinecone-client
!pip install llama-index-vector-stores-pinecone


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 215.9/215.9 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 56.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.1 MB/s eta 0:00:00


In [3]:
import pinecone
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, StorageContext
from llama_index.vector_stores.pinecone import PineconeVectorStore

## Create the Index in Pinecone

In [5]:
import os
from pinecone import Pinecone, ServerlessSpec

os.environ["PINECONE_API_KEY"] = "<your Pinecone API Key goes here>"
pc = Pinecone()

# This will create the index in Pinecone
pc.create_index(
    "rag", dimension=384,
    # "yoloxnet", dimension=384,
    metric="dotproduct",
    spec = ServerlessSpec(cloud="aws", region="us-east-1")
)

In [ ]:
from llama_index.core import SimpleDirectoryReader

# Load documents and build index
documents = SimpleDirectoryReader("./data").load_data()

# documents

## Construct VectorStore and customize StorageContext

In [ ]:
pinecone_index = pc.Index("yoloxnet")

vector_store = PineconeVectorStore(pinecone_index=pinecone_index)

storage_context = StorageContext.from_defaults(vector_store=vector_store)

## Set the Embedding Model to be used during creation of the vectors

In [ ]:
from llama_index.core import Settings
Settings.embed_model = embed_model

## Store Embeddings on Pinecone VectorDB

In [ ]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(
    documents,
    storage_context=storage_context
)

## Setup the LLM (Chatbot) to interact with using the Embeddings sored on Pinecone

In [20]:
# from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

Settings.llm = llm

## Query the LLM _ use both Query and Chat Mode

In [ ]:
# Formatted Display
from llama_index.core.response.notebook_utils import display_response

## General Knowledge

### Define a Query Engine

In [39]:
# Out of the context question

query_engine = index.as_query_engine(response_mode="compact")
# query_engine = index.as_query_engine()

# query_engine = index.as_query_engine(
#     response_mode="tree_summarize",
#     verbose=False,
# )

### Define a Chat Engine

In [ ]:
chat_engine = index.as_chat_engine()

In [ ]:
response = query_engine.query("How do OpenAI and Meta differ on AI tools?")
display_response(response)

In [ ]:
response = chat_engine.chat("How do OpenAI and Meta differ on AI tools?")
display_response(response)

In [ ]:
# Out of the context question

response = query_engine.query("Who is Sherlock Holmes?")
display_response(response)

In [ ]:
response = chat_engine.chat("Who is Sherlock Holmes?")
display_response(response)

## Context Specific Queries (RAG)

In [ ]:
# Context specific question
# LLM answers using RAG

response = query_engine.query("Explain what is YoloXNet model?")
display_response(response)

In [ ]:
response = chat_engine.chat("Explain what is YoloXNet model?")
display_response(response)

--------------------------------------------------

In [ ]:
response = query_engine.query("How does YoloXNet compare with Xception model?")
display_response(response)

In [ ]:
response = chat_engine.chat("How does YoloXNet compare with Xception model?")
display_response(response)

-------------------------------------------

In [ ]:
# Context specific question
# LLM answers using RAG

response = query_engine.query("How does YoloXNet differ from ResNet?")
display_response(response)

In [ ]:
response = chat_engine.chat("How does YoloXNet differ from ResNet?")
display_response(response)

In [ ]:
response = query_engine.query("Where is Heritage Institute of Technology located?")
display_response(response)

In [ ]:
response = chat_engine.chat("Where is Heritage Institute of Technology located?")
display_response(response)

----------------------------

In [ ]:
response = query_engine.query("Who is Shah Rukh Khan?")
display_response(response)

In [ ]:
response = chat_engine.chat("Who is Shah Rukh Khan?")
display_response(response)

## RAG is better than LLM-Hallucination

In [ ]:
response = query_engine.query("Explain why GPUs are not suitable fo training very large deep learing models?")
display_response(response)

In [ ]:
response = chat_engine.chat("Explain why GPUs are not suitable fo training very large deep learing models?")
display_response(response)

---------------------------------------------------------

#**Part:4: Some of Advanced Features**

# **Custom Response Synthesis**

In [ ]:
# Successfully installs.....
!pip install llama_index
!pip install llama-index-llms-anthropic

In [10]:
from llama_index.llms.anthropic import Anthropic
from llama_index.core import Settings

tokenizer = Anthropic().tokenizer
Settings.tokenizer = tokenizer

import os
os.environ["ANTHROPIC_API_KEY"] = "<your Anthropic API Key goes here>"

from llama_index.llms.anthropic import Anthropic

llm = Anthropic(model="claude-3-opus-20240229")

Settings.llm = llm


In [11]:
from llama_index.core import SimpleDirectoryReader

documents = SimpleDirectoryReader("/content/data").load_data()
text = documents[0].text


In [12]:
from llama_index.core.response_synthesizers import TreeSummarize
from llama_index.core.types import BaseModel
from typing import List

## Create pydantic model to structure response
class DLModel(BaseModel):
    """Data model for a DL model"""
    name: str
    key_features: List[str]
    extra_info: str


In [13]:
summarizer = TreeSummarize(verbose=True, output_cls=DLModel)

response = summarizer.get_response("What is YoloXNet model?", [text])

1 text chunks after repacking


/usr/local/lib/python3.10/dist-packages/llama_index/core/program/utils.py:58: UserWarning: Failed to use `OpenAIPydanticProgram`. Please ensure that is installed by running `pip install llama-index-program-openai`.
  warnings.warn(


In [14]:
print(response)

name='YoloXNet' key_features=['Hybrid model combining a deep convolutional neural network (CNN) and a Long Short-Term Memory (LSTM) network', 'CNN component extracts visual features from input images', 'LSTM component generates coherent and contextually relevant captions based on the visual features', 'Outperforms established models for image caption generation in quantitative metrics like BLEU scores', 'Captures nuanced details and contextual information in images', 'Demonstrates robustness in handling challenging scenarios with complex compositions or ambiguous subjects'] extra_info='YoloXNet is a cutting-edge deep learning model designed for image caption generation. It leverages the strengths of both CNNs for visual feature extraction and LSTMs for generating descriptive captions. Experiments show YoloXNet consistently outperforms existing models in generating high-quality, insightful captions for a wide range of images.'


In [15]:
print(response.name)

YoloXNet


In [16]:
response.key_features

['Hybrid model combining a deep convolutional neural network (CNN) and a Long Short-Term Memory (LSTM) network',
 'CNN component extracts visual features from input images',
 'LSTM component generates coherent and contextually relevant captions based on the visual features',
 'Outperforms established models for image caption generation in quantitative metrics like BLEU scores',
 'Captures nuanced details and contextual information in images',
 'Demonstrates robustness in handling challenging scenarios with complex compositions or ambiguous subjects']

In [17]:
response.extra_info

'YoloXNet is a cutting-edge deep learning model designed for image caption generation. It leverages the strengths of both CNNs for visual feature extraction and LSTMs for generating descriptive captions. Experiments show YoloXNet consistently outperforms existing models in generating high-quality, insightful captions for a wide range of images.'

# **Citation**

In [40]:
from llama_index.core.query_engine import CitationQueryEngine

query_engine = CitationQueryEngine.from_args(
    index,
    # increase the citation chunk size!
    citation_chunk_size= 256,  # 1024
    similarity_top_k=2,
)

# **Evaluate Response**

In [41]:
import nest_asyncio
nest_asyncio.apply()

In [42]:
# query
query = "What is YoloXNet?"
response = query_engine.query(query)
# response.response

##  FaithfulnessEvaluator - Solution to Hallucination

In [43]:
# The FaithfulnessEvaluator evaluates if the answer is faithful to the retrieved contexts (in other words, whether if there's hallucination).

from llama_index.core.evaluation import FaithfulnessEvaluator
# define evaluator
evaluator = FaithfulnessEvaluator(llm=llm)
eval_result = evaluator.evaluate_response(response=response)

print(str(eval_result.passing))



True


In [47]:
# print(eval_result.query)

print(eval_result.response)

print(eval_result.contexts)
print(eval_result.feedback)
print(eval_result.score)


According to the provided sources, YoloXNet is a hybrid deep learning model that combines a convolutional neural network (CNN) with a Long Short-Term Memory (LSTM) network. It uses the CNN as a visual encoder to extract features from input images and the LSTM network as a textual decoder to generate descriptive captions based on the visual cues [1]. YoloXNet excels in the task of image caption generation, providing rich and contextually relevant descriptions of images by leveraging both visual and textual modalities [2]. It showcases robustness in handling diverse visual stimuli and challenging scenarios, thanks to its hierarchical representation learned by the CNN and LSTM components [4].
['Source 1:\nA Comparison between YoloXNet, Xception, and ResNet \nAngela Yu, Yasik Dimitriov \nUniversity of Burgundy \n \nYoloXNet, Xception, and ResNet are all inﬂuential models in the realm of deep learning, each \ncontributing unique advancements to the ﬁeld of computer vision. Let\'s delve into

##  Response Relevancy

In [52]:
"""
Evaluating Query + Response Relevancy#
The RelevancyEvaluator evaluates if the retrieved context and the answer is relevant and consistent for the given query.

Note that this evaluator requires the query to be passed in, in addition to the Response object.
"""


from llama_index.core.evaluation import RelevancyEvaluator
# define evaluator
evaluator = RelevancyEvaluator(llm=llm)

eval_result = evaluator.evaluate_response(query=query, response=response)


In [46]:
print(eval_result.query)

print(eval_result.response)

print(eval_result.contexts)
print(eval_result.feedback)
print(eval_result.score)


What is YoloXNet?
According to the provided sources, YoloXNet is a hybrid deep learning model that combines a convolutional neural network (CNN) with a Long Short-Term Memory (LSTM) network. It uses the CNN as a visual encoder to extract features from input images and the LSTM network as a textual decoder to generate descriptive captions based on the visual cues [1]. YoloXNet excels in the task of image caption generation, providing rich and contextually relevant descriptions of images by leveraging both visual and textual modalities [2]. It showcases robustness in handling diverse visual stimuli and challenging scenarios, thanks to its hierarchical representation learned by the CNN and LSTM components [4].
['Source 1:\nA Comparison between YoloXNet, Xception, and ResNet \nAngela Yu, Yasik Dimitriov \nUniversity of Burgundy \n \nYoloXNet, Xception, and ResNet are all inﬂuential models in the realm of deep learning, each \ncontributing unique advancements to the ﬁeld of computer vision.

# **Pandas Query Engine**

In [61]:
!pip install llama-index llama-index-experimental

In [54]:
import logging
import sys
from IPython.display import Markdown, display

import pandas as pd
from llama_index.experimental.query_engine import PandasQueryEngine

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))


In [55]:
# Test on some sample data
df = pd.DataFrame(
    {
        "city": ["Toronto", "Tokyo", "Berlin"],
        "population": [2930000, 13960000, 3645000],
    }
)

df

,city,population
0,Toronto,2930000
1,Tokyo,13960000
2,Berlin,3645000


In [56]:
query_engine = PandasQueryEngine(df=df, verbose=True)


In [58]:
response = query_engine.query("What is the city with the highest population?")

display(Markdown(f"{response}"))


> Pandas Instructions:
```
df.loc[df['population'].idxmax(), 'city']
```
> Pandas Output: Tokyo


Tokyo

In [59]:
# get pandas python instructions
print(response.metadata["pandas_instruction_str"])

df.loc[df['population'].idxmax(), 'city']


In [60]:
query_engine = PandasQueryEngine(df=df, verbose=True, synthesize_response=True)

response = query_engine.query(
    "What is the city with the highest population? Give both the city and population",
)
print(str(response))


> Pandas Instructions:
```
df.loc[df['population'].idxmax(), ['city', 'population']].to_dict()
```
> Pandas Output: {'city': 'Tokyo', 'population': 13960000}
The city with the highest population is Tokyo, with a population of 13,960,000.
